# Extremely Noisy 4-D STEM Strain Mapping Using Cycle Consistent Spatial Transforming Autoencoders (Simulated 4dstem)

## Package Installation

In [1]:
import sys
!{sys.executable} -m pip install -i https://test.pypi.org/simple/ Auto4DSTEM==0.7.1

Looking in indexes: https://test.pypi.org/simple/


## Package Loading

In [2]:
%load_ext autoreload
%autoreload 2
import os
import torch
import pickle
import warnings
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from cmcrameri import cm
from auto4dstem.nn.Train_Function import TrainClass
from auto4dstem.Viz.util import download_files_from_txt,download_files_from_txt,mask_class
from auto4dstem.Viz.viz import set_format_Auto4D, visualize_simulate_result, visual_performance_plot,normalized_strain_matrices

## Device Setting

In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using GPU")
else:
    device = torch.device('cpu')
    print("Using CPU")

Using CPU


## Visualization Format 

In [4]:
set_format = set_format_Auto4D()
pylab.rcParams.update(set_format)

In [5]:
warnings.filterwarnings('ignore') 

## Download Data and Pre-Trained Models

In [6]:
folder_name = 'Extremely_Noisy_4DSTEM_Strain_Mapping_Using_CC_ST_AE_Simulated'
file_download =  "simulated_label_weights_affine_para_simple"

In [7]:
# check where are you from the root path
download_files_from_txt(file_download,folder_name)

 33%|███▎      | 1/3 [00:00<00:01,  1.13it/s]

File already exists: 25Percent_pretrained_rotation.npy


 67%|██████▋   | 2/3 [00:01<00:00,  1.18it/s]

File already exists: 25percent_noisy_simulated_4dstem_pretrained_weights.pkl


100%|██████████| 3/3 [00:02<00:00,  1.21it/s]

File already exists: polycrystal_output4D.mat


## Build mask for training and generating results

### Build Mask

In [8]:
# set mask class
set_mask = mask_class()
# generate mask 
#mask_tensor, mask_list = set_mask.mask_ring(radius_1=50, radius_2 = 85)
mask_tensor, mask_list = set_mask.mask_round(radius=7, center_list=[(99,162),(154,130),(99,36),(154,68),(45,68),(45,130)])

## Benchmark Simulated 4DSTEM Dataset

### Set BKG level, Load Data and Pre-Trained Weights

In [9]:
bkg_intensity = 0.25
bkg_str = format(int(bkg_intensity*100),'02d')
data_path = os.path.abspath(f'{folder_name}/polycrystal_output4D.mat')
rotation_path = f'{folder_name}/{bkg_str}Percent_pretrained_rotation.npy'

## Initialize the training class from noise-free simulated dataset

In [10]:
rotation_ = np.load(rotation_path)

In [11]:
rotation_.shape

(65536, 2)

In [12]:
import h5py

In [13]:
# f = h5py.File(data_path,'r')
# op4d = f['output4D']
# test_data = op4d[0:16,0:16]
# np.save('test_4dstem_16x16.npy',test_data)

In [14]:
# test_rotation = rotation_[0:16*16]
# np.save('test_rotation_256.npy',test_rotation)

In [15]:
tc = TrainClass(data_path,
                device = device, 
                transpose = (1,0,3,2),
                background_weight=bkg_intensity,
                learned_rotation=rotation_path,
                adjust_learned_rotation = -15,
                num_base=1,
                up_size=800,
                scale_limit=0.05,
                shear_limit=0.1,
                rotation_limit=0.1,
                trans_limit = 0.15,
                adj_mask_para=0,
                num_mask = 6,
                fixed_mask = mask_list,
                check_mask = None,
                interpolate = True,
                revise_affine = True,
                folder_path = folder_name
                )

/Users/chenjiaqing/Documents/GitHub/Auto4DSTEM/Extremely_Noisy_4DSTEM_Strain_Mapping_Using_CC_ST_AE_Simulated/polycrystal_output4D.mat
add Poison distributed background noise to whole dataset


100%|██████████| 65536/65536 [01:45<00:00, 620.58it/s]


add image-rotation pair to whole dataset


100%|██████████| 65536/65536 [00:00<00:00, 182174.56it/s]


## Test Training Process

In [16]:
tc.folder_path = 'test_loss_function'

In [17]:
tc.data_set.shape

(65536, 1, 200, 200)

In [18]:
tc.batch_size = 16
tc.train_process()

  1%|          | 41/4096 [14:30<24:08:27, 21.43s/it]